In [1]:
from commons import *

In [2]:
# Generate Map Size
max_x = 200
max_y = 200
curr_points = {}
WorldMap = Map(max_x, max_y, curr_points)

***self.points {}


In [3]:
# Initialize PersonIds object
# curr_pc_Ids = {}
# curr_npc_Ids = {}
# curr_item_Ids = {}
# curr_path_Ids = {}
# curr_obstacle_Ids = {}

# Object holding all ids with separate categories
ids = Ids()

# Test ids object is empty
ids.npc_Ids.ids

In [4]:
# Populate Map with Enemies

# Create a Skeleton in point(101,101)
starting_pos = (101,101)
pointCategory = PointCategories.npc
name = 'Skelly'
skeleton = create_default_person(name, starting_pos, 'skeleton', WorldMap, pointCategory, ids)
skeleton.curr_pos.point

PointCategories.npc
***mapPoint <commons.PointCategoriesObj object at 0x000001534F38DB48>
***add_to_points (101, 101) npc 0


(101, 101)

In [5]:
# Create Main Character
starting_pos = (100,100)
pointCategory = PointCategories.pc
name = 'Danny'
main_char = create_default_person(name, starting_pos, 'main_character', WorldMap, pointCategory, ids)
main_char.curr_pos.point

PointCategories.pc
***mapPoint <commons.PointCategoriesObj object at 0x000001534F1C9CC8>
***add_to_points (100, 100) pc 0


(100, 100)

In [6]:
# Test Move Character to Skeleton
# Right now, movement is working, next step is to build the Encounter class
movement = (1,1) # Can move in 8 ways, diagonally (4) and up/down left/right (4)
is_encounter = main_char.move_spaces(movement, WorldMap)
print(main_char.curr_pos.point)
if is_encounter:
    print('ENCOUNTER')

++
***add_to_points (101, 101) pc 0
(101, 101)
ENCOUNTER


In [7]:
# print(WorldMap.points[(101,101)].pointCategories['pc'].head.data)
# print(WorldMap.points[(101,101)].pointCategories['pc'].head.next)
# print(WorldMap.points[(101,101)].pointCategories['pc'].head.next.next)

print('***')
print(WorldMap.points[(101,101)].pointCategories['npc'].head.data.id)
print(WorldMap.points[(101,101)].pointCategories['npc'].head.next.data.id)
print(WorldMap.points[(101,101)].pointCategories['npc'].head.next.next.data.id)
print(WorldMap.points[(101,101)].pointCategories['npc'].head.next.next.next.data.id)

***
0


AttributeError: 'NoneType' object has no attribute 'data'

In [8]:
for key in WorldMap.points.keys():
    print(key)
    print('*pc')
    WorldMap.points[key].pointCategories['pc'].printList()
    print('*npc')
    WorldMap.points[key].pointCategories['npc'].printList()

(101, 101)
*pc
*npc
(100, 100)
*pc
*npc
